# <center>Minimum Transformation Method
## <center>An inverse model for inferring surface fluxes and mixing from gridded hydrographic TS data, <br> by Taimoor Sohail and Jan D. Zika



##### <center>We use an optimal transport method, minimzing a cost function which respects the volume and mass conservation, and the fundamental physics of mixing in TS space. <br>See also: Jan D Zika et al. (2021) [www.doi.org/10.1175/JCLI-D-20-0355.1 ] for a simpler version of this model which optimizes ocean circulation only.
------------

### __The code is laid out as follows:__
#### A) Pre-processing
i) Load necessary modules for computation \
ii) Define key parameters
#### B) Load Data
#### C) Define constraints
i) Define Connectivity \
ii) Define weights
#### D) Run optimisation
#### E) Save Output
---------

## A) Pre-processing

### i) Load necessary modules for computation

In [6]:
## Module to run the minimisation
from WM_Methods import MTM
## Module to track runtime of cells and loops
import time
from tqdm.notebook import tqdm
## Suppress warnings related to division by zero
import warnings
warnings.filterwarnings('ignore')
## Module to load files and handle array computations
import xarray as xr
import numpy as np
## Modules to plot outputs
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import gc
# Specify font properties for plots
# from matplotlib import rc
# font = {'family' : 'DejaVu Sans',
#         'weight' : 'normal',
#         'size'   : 16}

# rc('font', **font)

### ii) Define key parameters

In [7]:
# Thermal expansion, haline contraction and true scale factor
alph = 1.7657*10**-4
bet = 7.5544*10**-4

volnorming = 10**15 #normalising coeffcients
areanorming = 10**12 #normalising coeffcients
ST_scale=bet/alph

# Establish basic constants 
yr2sec = 365.25*24*60*60
Cp=4000
rho=1024
S0=35

# If Surface fluxes are available
SF = True
# To run the optimisation twice 
Opt_twice = False

# Range of years of which 'early' and 'late' are defined
for z in tqdm(range(36)):
    ## ERA5 begins in 1979 so we have to change our "early" and "late" periods
    dyrs = 0
    init_early = 1979+z
    init_late = 1979+z
    
    Early_period = (np.array([z,z+1]))*12
    Late_period = (np.array([z,z+1]))*12
    range_yrs = init_late-init_early+1

    print(Early_period,Late_period)
    
    ERA5_BSP_data = xr.open_mfdataset('BSP_processed/BSP_ERA5_TS_*.nc')
    
    ## Early Period
    Part_early = ERA5_BSP_data.Partitions.isel(Time=slice(Early_period[0],Early_period[1])).mean('Time')
    SA_early =  ERA5_BSP_data.S_mean.isel(Time=slice(Early_period[0],Early_period[1])).mean('Time')
    CT_early = ERA5_BSP_data.T_mean.isel(Time=slice(Early_period[0],Early_period[1])).mean('Time')
    V_early = ERA5_BSP_data.V_sum.isel(Time=slice(Early_period[0],Early_period[1])).mean('Time')
    A_early = ERA5_BSP_data.A_sum.isel(Time=slice(Early_period[0],Early_period[1])).mean('Time')

    ## Late Period
    Part_late = ERA5_BSP_data.Partitions.isel(Time=slice(Late_period[0],Late_period[1])).mean('Time')
    SA_late =  ERA5_BSP_data.S_mean.isel(Time=slice(Late_period[0],Late_period[1])).mean('Time')
    CT_late = ERA5_BSP_data.T_mean.isel(Time=slice(Late_period[0],Late_period[1])).mean('Time')
    V_late = ERA5_BSP_data.V_sum.isel(Time=slice(Late_period[0],Late_period[1])).mean('Time')
    A_late = ERA5_BSP_data.A_sum.isel(Time=slice(Late_period[0],Late_period[1])).mean('Time')

    Basins = ERA5_BSP_data.Basin.values

    time = ERA5_BSP_data.Time.values
    ## Calculate the time-mean hfds and wfo
    dhfds = ERA5_BSP_data.hfds_sum.isel(Time=slice(Late_period[0],Late_period[1])).mean('Time')*(1/(24*3600)) # units: W
    dwfo = ERA5_BSP_data.wfo_sum.isel(Time=slice(Late_period[0],Late_period[1])).mean('Time') # units: kg/s

    ## Convert dflux to equivalent T or S change
    dT_hfds = dhfds/(Cp*rho*V_early)*yr2sec # units: C/yr
    dS_wfo = -dwfo*S0/(rho*V_early)*yr2sec # units: g/kg/yr

    # Flatten the early and late variables to a 1D array
    Vol_1 = V_early.values.flatten()
    Vol_2 = V_late.values.flatten()
    S_1 = SA_early.values.flatten()-S0 # Remove reference salinity S0
    S_2 = SA_late.values.flatten()-S0 # Remove reference salinity S0
    T_1 = CT_early.values.flatten()
    T_2 = CT_late.values.flatten()
    A_1 = A_early.values.flatten()
    A_2 = A_late.values.flatten()

    # Do the same for basin index
    Basin_1 = np.zeros_like(V_early)
    Basin_2 = np.zeros_like(V_early)
    Basin_names = []
    for i in range(np.array(Basins).size):
        Basin_1[i,:] = i
        Basin_2[i,:] = i
        for j in range(V_early.shape[-1]):
            #... and for basin name
            Basin_names.append(Basins[i])

    Basin_1_inds = Basin_1.flatten()
    Basin_2_inds = Basin_2.flatten()

    #... and for the edges of the BSP bins
    ## Here we calculate the mean TS edges averaged over both early and late times
    S_start = (0.5*(Part_early.values[:,:,0]+Part_late.values[:,:,0])).flatten()-S0 # Remove reference salinity S0
    S_end = (0.5*(Part_early.values[:,:,1]+Part_late.values[:,:,1])).flatten()-S0 # Remove reference salinity S0
    T_start = (0.5*(Part_early.values[:,:,2]+Part_late.values[:,:,2])).flatten()
    T_end = (0.5*(Part_early.values[:,:,3]+Part_late.values[:,:,3])).flatten()

    # Any NaNs are zeroed out
    S_1[np.isnan(S_1)] = 0
    S_2[np.isnan(S_2)] = 0
    T_1[np.isnan(T_1)] = 0
    T_2[np.isnan(T_2)] = 0

    ## IF SURFACE FLUXES ARE PROVIDED -- add here ##
    S_pre = SA_early.values.flatten()-S0
    S_1 = SA_early.values.flatten()-S0+dS_wfo.values.flatten()
    T_pre = CT_early.values.flatten()
    T_1 = CT_early.values.flatten()+dT_hfds.values.flatten()

    S_1[np.isnan(S_1)] = 0
    T_1[np.isnan(T_1)] = 0
    S_1[~np.isfinite(S_1)] = 0
    T_1[~np.isfinite(T_1)] = 0

    ## Here, we create the tracers and volumes matrices, which will be fed into the MTM function

    volumes = np.stack((Vol_1, Vol_2), axis=0)/volnorming # Shape: [2 x N]

    salinities = np.stack((S_1, S_2), axis=0)
    temps = np.stack((T_1, T_2), axis=0)

    tracers = np.stack((salinities, temps),axis=1) # Shape: [2 x M x N], where M = 2 for just T and S, and M>2 for T,S+other tracers

    print('Total number of bins =', int(Vol_1.shape[0]))
    N = int(Vol_1.shape[0])

    # Array defining the connection between the 9 basins;
    # 1 = connected, 0 = disconnected
    connectivity_array = np.zeros((Basins.size,Basins.size))

    connectivity_array[0,:] = [1, 1, 0, 0, 0, 0, 0, 0, 0]
    connectivity_array[1,:] = [1, 1, 1, 0, 0, 0, 0, 0, 0]
    connectivity_array[2,:] = [0, 1, 1, 1, 0, 0, 0, 0, 0]
    connectivity_array[3,:] = [0, 0, 1, 1, 0, 0, 0, 0, 1]
    connectivity_array[4,:] = [0, 0, 0, 0, 1, 0, 1, 0, 1]
    connectivity_array[5,:] = [0, 0, 0, 0, 0, 1, 1, 0, 1]
    connectivity_array[6,:] = [0, 0, 0, 0, 1, 1, 1, 1, 0]
    connectivity_array[7,:] = [0, 0, 0, 0, 0, 0, 1, 1, 0]
    connectivity_array[8,:] = [0, 0, 0, 1, 1, 1, 0, 0, 1]

    # Array defining the transport between the 9 basins;
    # +/-1 = connected (North = +, East = +), 0 = no constraint
    transport_array = np.zeros((Basins.size,Basins.size))

    transport_array[4,:] = [0, 0, 0, 0, 0, 0, 1, 0, 0]
    transport_array[6,:] = [0, 0, 0, 0, -1, 0, 0, 0, 0]

    print(Basins)

    # Define whether a bin is connected to every other bin
    # The two constraints used are: are the basins adjacent? 
    # If yes, are the bin indices the same? 
    # If yes, the bins are connected; if no, they are not connected. 
    connected = np.zeros((Vol_1.size, Vol_1.size))
    trans_big = np.zeros((Vol_1.size, Vol_1.size))

    for i in tqdm(range(Vol_1.size)):
        for j in range(Vol_2.size):
            trans_big[i,j] = transport_array[int(Basin_1_inds[i]), int(Basin_2_inds[j])]
            if connectivity_array[int(Basin_1_inds[i]), int(Basin_2_inds[j])]>0:
                    if Basin_names[i] == Basin_names[j]:
                        connected[i,j] = connectivity_array[int(Basin_1_inds[i]), int(Basin_2_inds[j])]
                    elif S_start[i]==S_start[j] and T_start[i]==T_start[j]:
                        connected[i,j] = connectivity_array[int(Basin_1_inds[i]), int(Basin_2_inds[j])]

    constraints = connected # Shape: An [N x N] matrix

    transport = trans_big

    ## We create a weight matrix
    area_weight = np.log10(areanorming)/(np.log10(A_2))
    area_weight[area_weight==0] = 10
    weights = np.stack((ST_scale*area_weight,area_weight), axis=0) # Shape: An [M x N] matrix

    ## We run the optimiser to get the transports between water masses and the T,S mixed and T,S adjustment
    result = MTM.optimise(tracers, volumes, constraints, transport, weights)

    g_ij = result['g_ij'] ## An [N x N] matrix of transports between WMs
    Mixing = result['Mixing'] ## An [M x N] matrix of dtracer mixing for each WM
    Adjustment = result['Adjustment'] ## An [M x N] matrix of dtracer adjustment for each WM

    ## Break down the Mixing and Adjustment matrices into their constituent tracers
    dT_mixing = Mixing[1,:]
    dS_mixing = Mixing[0,:]
    dS_adj = Adjustment[0,:]
    dT_adj = Adjustment[1,:]

    da_dT_mixing = xr.DataArray(data = dT_mixing, dims = ["WM_number"],
                           coords=dict(WM_number = np.arange(0,N)),
                        attrs=dict(description="Temperature Mixing", units="\Delta K", variable_id="EN4 Tmix"))
    da_dS_mixing = xr.DataArray(data = dS_mixing, dims = ["WM_number"],
                            coords=dict(WM_number = np.arange(0,N)),
                            attrs=dict(description="Salinity Mixing", units="\Delta g/kg", variable_id="EN4 Smix"))
    da_dT_adjustment = xr.DataArray(data = dT_adj, dims = ["WM_number"],
                            coords=dict(WM_number = np.arange(0,N)),
                            attrs=dict(description="Temperature Adjustment", units="\Delta K", variable_id="EN4 Tadj"))
    da_dS_adjustment = xr.DataArray(data = dS_adj, dims = ["WM_number"],
                            coords=dict(WM_number = np.arange(0,N)),
                            attrs=dict(description="Salinity Adjustment", units="\Delta g/kg", variable_id="EN4 Sadj"))
    da_gij = xr.DataArray(data = g_ij*volnorming, dims = ["WM_initial", "WM_final"],
                            coords=dict(WM_initial = np.arange(0,N), WM_final = np.arange(0,N)),
                            attrs=dict(description="Volume transport", units="m^3", variable_id="EN4 Gij"))

    ## Create xarray DataSet that will hold all these DataArrays
    ds_BSP = xr.Dataset()
    ds_BSP['dT_mixing'] = da_dT_mixing
    ds_BSP['dS_mixing'] = da_dS_mixing
    ds_BSP['dT_adjustment'] = da_dT_adjustment
    ds_BSP['dS_adjustment'] = da_dS_adjustment
    ds_BSP['gij'] = da_gij

    ds_BSP.to_netcdf('Optimisation_results/Optimal_result_EN4_trend_%i.nc' %(z), mode='w')


  0%|          | 0/27 [00:00<?, ?it/s]

[204 216] [204 216]
Total number of bins = 1152
['Polar N. Atlantic' 'Subtropical N. Atlantic' 'Eq. Atlantic'
 'S. Atlantic' 'Indian' 'S. Pacific' 'Eq. Pacific' 'N. Pacific'
 'Southern Ocean']


  0%|          | 0/1152 [00:00<?, ?it/s]

                                     CVXPY                                     
                                    v1.1.13                                    
(CVXPY) Jun 17 11:24:20 PM: Your problem has 149760 variables, 3 constraints, and 0 parameters.
(CVXPY) Jun 17 11:24:25 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jun 17 11:24:25 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jun 17 11:24:25 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jun 17 11:24:25 PM: Compiling problem (target solver=MOSEK).
(CVXPY) Jun 17 11:24:25 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffi

  0%|          | 0/1152 [00:00<?, ?it/s]

...

  0%|          | 0/1152 [00:00<?, ?it/s]

                                     CVXPY                                     
                                    v1.1.13                                    
(CVXPY) Jun 17 11:46:58 PM: Your problem has 149760 variables, 3 constraints, and 0 parameters.
(CVXPY) Jun 17 11:47:01 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jun 17 11:47:01 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jun 17 11:47:01 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jun 17 11:47:01 PM: Compiling problem (target solver=MOSEK).
(CVXPY) Jun 17 11:47:01 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffi

  0%|          | 0/1152 [00:00<?, ?it/s]

                                     CVXPY                                     
                                    v1.1.13                                    
(CVXPY) Jun 17 11:48:10 PM: Your problem has 147456 variables, 3 constraints, and 0 parameters.
(CVXPY) Jun 17 11:48:13 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jun 17 11:48:13 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jun 17 11:48:13 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jun 17 11:48:13 PM: Compiling problem (target solver=MOSEK).
(CVXPY) Jun 17 11:48:13 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffi

Error: rescode.err_nan_in_c(1470): The objective vector c contains an invalid value for variable '' (0).